In [147]:
import pandas as pd
import altair as alt

In [119]:
outages = pd.read_csv('data/outages.csv', parse_dates=['START', 'END'])

In [120]:
len(outages)

1796

In [121]:
outages.head()

,STATION,STATION LONG NAME,UNIT NAME,SEASONAL MAX MW (HSL),AVAILABLE MW AFTER OUTAGE/DERATE,MW REDUCTION FROM OUTAGE/DERATE,FUEL TYPE,START,END,RESOURCE ENTITY,COUNTY
0,HHGT,HORSE HOLLOW GENERATION TIE,HHOLLOW3,224.0,5.0,219.0,WIND,2021-02-14 00:00:00,2021-02-15 23:59:00,HORSE HOLLOW GENERATION TIE LLC 3 (RE),TAYLOR
1,WAP,WA PARISH,WAP_G2,169.0,43.0,126.0,NG,2021-02-14 00:00:00,2021-02-14 02:00:00,NRG TEXAS POWER LLC (RE),FORT BEND
2,WAP,WA PARISH,WAP_G4,552.0,276.0,276.0,NG,2021-02-14 00:00:00,2021-02-14 01:00:00,NRG TEXAS POWER LLC (RE),FORT BEND
3,KARAKAW1,KARANKAWA1 WIND FARM,UNIT2,103.0,83.0,20.0,WIND,2021-02-14 00:30:00,2021-02-14 15:50:00,KARANKAWA WIND LLC B (RE),SAN PATRICIO
4,BULLCRK,BULL CREEK WIND,WND1,88.0,0.0,88.0,WIND,2021-02-14 00:43:00,2021-02-16 01:48:00,BULL CREEK WIND LLC (RE),BORDEN


In [122]:
outages['FUEL TYPE'].unique()

array(['WIND', 'NG', nan, 'SOLAR', 'COAL', 'ESR', 'HYDRO', 'NUCLEAR'],
      dtype=object)

In [123]:
outages['START'].min(), outages['END'].max()

(Timestamp('2021-02-14 00:00:00'), Timestamp('2021-12-17 15:31:00'))

In [133]:
gas = outages['FUEL TYPE'] == 'NG'
started = outages['START'] <= '2021-02-14 00:00'
not_ended = outages['END'] > '2021-02-14 00:00'
outages[gas & started & not_ended]

,STATION,STATION LONG NAME,UNIT NAME,SEASONAL MAX MW (HSL),AVAILABLE MW AFTER OUTAGE/DERATE,MW REDUCTION FROM OUTAGE/DERATE,FUEL TYPE,START,END,RESOURCE ENTITY,COUNTY
1,WAP,WA PARISH,WAP_G2,169.0,43.0,126.0,NG,2021-02-14,2021-02-14 02:00:00,NRG TEXAS POWER LLC (RE),FORT BEND
2,WAP,WA PARISH,WAP_G4,552.0,276.0,276.0,NG,2021-02-14,2021-02-14 01:00:00,NRG TEXAS POWER LLC (RE),FORT BEND


In [134]:
fuels = pd.Series({
    'WIND': 'wind',
    'NG': 'gas',
    'SOLAR': 'solar',
    'COAL': 'coal',
    'NUCLEAR': 'nuke',
}, name='fuel')
fuels

WIND        wind
NG           gas
SOLAR      solar
COAL        coal
NUCLEAR     nuke
Name: fuel, dtype: object

In [135]:
# Normalize the fuel name and select just the fuels whose capacity we know.
outages = pd.merge(left=outages, right=fuels, left_on='FUEL TYPE', right_index=True, how='inner')

In [136]:
# Hard to get good data for this.
# ERCOT last disclosed facilities in December 2020, but capacity has changed since then.
# They summarized only the biggest capacity fuel types in their report to Legislature.
# I'm leaning toward the most up-to-date data, even if it leaves out smaller fuel types.
capacity = pd.read_csv('data/capacity.csv').set_index('fuel')
capacity

,mw
fuel,
gas,51667
coal,13630
nuke,5153
wind,31390
solar,6177


In [161]:
times = pd.date_range(start='2021-02-14 00:00', end='2021-02-19 10:00', freq='15min', name='time')
times = times.to_frame().reset_index(drop=True)
len(times)

521

In [162]:
times['key'] = 0
outages['key'] = 0
joined = pd.merge(left=times, right=outages, how='outer', on='key')
len(joined)

858608

In [163]:
joined.head()

,time,key,STATION,STATION LONG NAME,UNIT NAME,SEASONAL MAX MW (HSL),AVAILABLE MW AFTER OUTAGE/DERATE,MW REDUCTION FROM OUTAGE/DERATE,FUEL TYPE,START,END,RESOURCE ENTITY,COUNTY,fuel
0,2021-02-14,0,HHGT,HORSE HOLLOW GENERATION TIE,HHOLLOW3,224.0,5.0,219.0,WIND,2021-02-14 00:00:00,2021-02-15 23:59:00,HORSE HOLLOW GENERATION TIE LLC 3 (RE),TAYLOR,wind
1,2021-02-14,0,KARAKAW1,KARANKAWA1 WIND FARM,UNIT2,103.0,83.0,20.0,WIND,2021-02-14 00:30:00,2021-02-14 15:50:00,KARANKAWA WIND LLC B (RE),SAN PATRICIO,wind
2,2021-02-14,0,BULLCRK,BULL CREEK WIND,WND1,88.0,0.0,88.0,WIND,2021-02-14 00:43:00,2021-02-16 01:48:00,BULL CREEK WIND LLC (RE),BORDEN,wind
3,2021-02-14,0,BULLCRK,BULL CREEK WIND,WND2,90.0,0.0,90.0,WIND,2021-02-14 00:43:00,2021-02-16 01:48:00,BULL CREEK WIND LLC (RE),BORDEN,wind
4,2021-02-14,0,KARAKAW1,KARANKAWA1 WIND FARM,UNIT1,103.0,83.0,20.0,WIND,2021-02-14 00:45:00,2021-02-14 15:50:00,KARANKAWA WIND LLC B (RE),SAN PATRICIO,wind


In [164]:
started = joined['START'] <= joined['time']
not_ended = joined['END'] > joined['time']
ongoing = joined[started & not_ended]
ongoing.head()

,time,key,STATION,STATION LONG NAME,UNIT NAME,SEASONAL MAX MW (HSL),AVAILABLE MW AFTER OUTAGE/DERATE,MW REDUCTION FROM OUTAGE/DERATE,FUEL TYPE,START,END,RESOURCE ENTITY,COUNTY,fuel
0,2021-02-14 00:00:00,0,HHGT,HORSE HOLLOW GENERATION TIE,HHOLLOW3,224.0,5.0,219.0,WIND,2021-02-14,2021-02-15 23:59:00,HORSE HOLLOW GENERATION TIE LLC 3 (RE),TAYLOR,wind
767,2021-02-14 00:00:00,0,WAP,WA PARISH,WAP_G2,169.0,43.0,126.0,NG,2021-02-14,2021-02-14 02:00:00,NRG TEXAS POWER LLC (RE),FORT BEND,gas
768,2021-02-14 00:00:00,0,WAP,WA PARISH,WAP_G4,552.0,276.0,276.0,NG,2021-02-14,2021-02-14 01:00:00,NRG TEXAS POWER LLC (RE),FORT BEND,gas
1648,2021-02-14 00:15:00,0,HHGT,HORSE HOLLOW GENERATION TIE,HHOLLOW3,224.0,5.0,219.0,WIND,2021-02-14,2021-02-15 23:59:00,HORSE HOLLOW GENERATION TIE LLC 3 (RE),TAYLOR,wind
2415,2021-02-14 00:15:00,0,WAP,WA PARISH,WAP_G2,169.0,43.0,126.0,NG,2021-02-14,2021-02-14 02:00:00,NRG TEXAS POWER LLC (RE),FORT BEND,gas


In [168]:
tf = ongoing.groupby(['time', 'fuel'])['MW REDUCTION FROM OUTAGE/DERATE'].sum().reset_index()
tfc = pd.merge(left=tf, right=capacity, on='fuel')
tfc['factor'] = tfc['MW REDUCTION FROM OUTAGE/DERATE'] / tfc['mw']
# Have to do this pivot to get nice zeroes at times a fuel has no outage.
fuel_by_time = tfc.pivot(index='time', columns='fuel', values='factor').fillna(0)
fuel_by_time

fuel,coal,gas,nuke,solar,wind
time,,,,,
2021-02-14 00:00:00,0.000000,0.007781,0.0,0.00000,0.006977
2021-02-14 00:15:00,0.000000,0.007781,0.0,0.00000,0.006977
2021-02-14 00:30:00,0.000000,0.007781,0.0,0.00000,0.007614
2021-02-14 00:45:00,0.000000,0.007781,0.0,0.00000,0.013922
2021-02-14 01:00:00,0.000000,0.006542,0.0,0.00000,0.013922
...,...,...,...,...,...
2021-02-19 09:00:00,0.076449,0.168328,0.0,0.05569,0.304810
2021-02-19 09:15:00,0.076449,0.168328,0.0,0.05569,0.304810
2021-02-19 09:30:00,0.076449,0.159618,0.0,0.05569,0.304810


In [170]:
source = fuel_by_time.reset_index().melt(id_vars=['time'], value_vars=fuels.values, value_name='outage')
# Make it a percent for the laypeople.
source['outage'] *= 100
source

,time,fuel,outage
0,2021-02-14 00:00:00,wind,0.697674
1,2021-02-14 00:15:00,wind,0.697674
2,2021-02-14 00:30:00,wind,0.761389
3,2021-02-14 00:45:00,wind,1.392163
4,2021-02-14 01:00:00,wind,1.392163
...,...,...,...
2600,2021-02-19 09:00:00,nuke,0.000000
2601,2021-02-19 09:15:00,nuke,0.000000
2602,2021-02-19 09:30:00,nuke,0.000000
2603,2021-02-19 09:45:00,nuke,0.000000


In [172]:
alt.Chart(source).mark_line().encode(
    alt.X('time'),
    alt.Y('outage'),
    alt.Color('fuel'),
)

alt.Chart(...)